In [13]:
# Instalación de dependencias desde requirements.txt
import sys
!{sys.executable} -m pip install -r ../requirements.txt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from cargar_datos import CargarDatos

# Carga de datos integrada
df = CargarDatos()
df.head()


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\RuVe7\\Desktop\\ProyectoM5_MatiasGutierrez\\Pi_pt_ds_01\\mlops_pipeline\\src\\Base_de_datos.xlsx'

In [ ]:
# --- Descripción general de los datos ---
print("Resumen de la estructura:")
df.info()

# --- Caracterización de los datos (Consigna) ---
# Clasificación manual basada en la naturaleza del dato:
# Nominales: tipo_laboral, tendencia_ingresos
# Ordinales: tipo_credito (si tiene jerarquía)
# Numéricos: capital_prestado, salario_cliente, edad_cliente, puntaje...
# Dicotómicos: Pago_atiempo (0/1)
# Politómicos: tendencia_ingresos (Creciente, Decreciente, Estable)

# --- Unificar valores Nulos ---
# Reemplazamos posibles placeholders por NaN real de numpy
df = df.replace(['?', 'N/A', 'null', 'None', ''], np.nan)
print(f"\nNulos por columna:\n{df.isnull().sum()}")

# --- Eliminación de variables irrelevantes ---
# Por ejemplo, IDs o fechas que no aportan al modelo estadístico puro
df = df.drop(columns=['fecha_prestamo'], errors='ignore')

# --- Conversión de tipos de datos ---
# Aseguramos que las categorías sean categóricas y los números sean flotantes
cat_cols = ['tipo_laboral', 'tendencia_ingresos', 'Pago_atiempo']
for col in cat_cols:
    df[col] = df[col].astype('category')

num_cols = df.select_dtypes(include=[np.number]).columns
print(f"\nVariables numéricas procesadas: {list(num_cols)}")

In [4]:
# Instalación de todas las librerías desde el archivo requirements.txt
import sys
!{sys.executable} -m pip install -r requirements.txt


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
# Instalación de librerías necesarias
!pip install pandas numpy matplotlib seaborn openpyxl scipy

  Using cached matplotlib-3.10.8-cp313-cp313-win_amd64.whl.metadata (52 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached scipy-1.17.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp313-cp313-win_amd64.whl.metadata (116 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.7 MB 4.9 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.7 MB 5.4 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.7 MB 5.7 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.7 MB 5.8 MB/s eta 0:0


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis

# Importación del módulo personalizado
from cargar_datos import CargarDatos

# Carga de los datos
df = CargarDatos()

# Verificación inicial
print(f"Dataset cargado con {df.shape[0]} registros y {df.shape[1]} columnas.")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\RuVe7\\Desktop\\ProyectoM5_MatiasGutierrez\\Pi_pt_ds_01\\mlops_pipeline\\src\\Base_de_datos.xlsx'

### 2 Exporacion de datos

#### 2.1 Revision de nulos

In [9]:
df.info

NameError: name 'df' is not defined

Se eliminaran arquellos atributos que presenten un nivel muy alto de valores nulos, ya que requeririan un esfuerzo para imputarlos y podrian introducir sesgos en el analisis. En el caso de promedio_ingresos_datacreditos y tendencia_ingresos, ambas presentan alrededor del 30% de los valores faltantes. Si bien este porcentaje podria justificar su eliminacion, se ha decidido conservarlas temporalmente con el fin de evaluarlas en etapas posteriores y determinar si aportan valor predictivo o deben finalmente descartarse

2.1 TIPO DE VARIABLE

variables categoricas

ordinales

+ tipo_credito (politomico)
+ fecha_prestamo (politomico)

Nominales

+ tipo_laboral (Dicotomico)
+ tendencia_ingresos (politomico)

Variables numericas

Continuas

* capital_prestado
* salario_cliente
* total_otros_prestamos
* puntaje
* puntaje_datacredito
* saldo_mora
* saldo_total
* saldo_principal
* Promedio_ingresos_datacredito

Discretas

* plazo_meses
* edad_cliente
* couta_pactada
* cant_creditosvigentes
* huella_consulta
* creditos_sectorFinanciero
* creditos_sectorCooperativo
* creditos_sectorReal






#### 2.2 Unificar la forma como se representan los valores Nulos

In [ ]:
!pip install numpy

In [ ]:
import numpy as np
df = df.replace(["NA","N/A", "null", "NULL","?", "Sin dato"], np.nan)

#### Eliminacion de variables irrelevantes

In [ ]:
df = df.drop(columns=["fecha_prestamo", "plazo_meses")

No era relevante para los objetivos iniciales de la exploracion, con couta pactada y capital prestado se puede saber el plazo de pago
#### Conversion de tipode datos
Se convierten las variables categoricas tipo object a category

In [ ]:
df["tipo_credito"] = df["tipo_credito"].astype("category")

df["tipo_laboral"] = df["tipo_laboral"].astype("category")

df["tendencia_ingresos"] = df["tendencia_ingresos"].astype("category")


In [ ]:
df.info()

### EDA
#### Analisis Univariable
##### Para variables numericas usamos .describe

In [ ]:
#Variables numericas

df.describe()

In [ ]:
df_num = df.selectdtypes(include=["int64","float64"])
df_num.agg(["mean", "median", "std", "var", "min", "max", "skew", "kurt"])

Para las variables categoriacas un value_counts()

In [ ]:
df.tipo_laboral.value_counts()

In [ ]:
df.tipo_credito.value_counts()

Eliminamos los tipo credito con pocos registros

In [ ]:
df = df[df["tipo_credito"].isin([7,68])]
df["tipo_credito"]=df["tipo_credito"].cat.remove_unused_categories()

In [ ]:
df.tipo_credito.value_counts()

In [ ]:
df.tendencia_ingresos.value_counts()

Podemos ver que hay un problema de calidad de datso en la variable tendencia_ingresos por lo que tenemos que corregir esta variable dejando las caregorias esperadas (Creciente, decreciente, estable) y eliminar los datos numericos que no corresponden a una variable categorica

In [ ]:
categorias_validas = ["Creciente", "Decreciente", "Estable"]
df["tendencia_ingresos"] = df["tendencia_ingresos"].where(
    df["tendencia_ingresos"].isin(categorias_validas),
    np.nan
)

df["tendencia_ingresos"] = df["tendencia_ingresos"].astype("category")
df["tendencia_ingresos"] = df["tendencia_ingresos"].cat.remove_unused_categories()

La varieble tendencia_ingresos presentaba valores erroneos (numericos que no corresponden a su naturaleza categorica). Estos registros representan menos del 1% del total, por lo cual se reemplazaron por NaN. El total de categorias validas paso de 7831 a 7773 registros, manteniendo la coherencia de la variable.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#   Filtrar solo columnas numericas
num_cols = df.select_dytypes(include=['int64','float64']).columns

#   Recorrer y graficar histogramas
for col in num_cols:
    plt.figure(figsize=(8,4))
    sns.histplot(df[col].dropna(), bins=30)
    plt.title(f"Histograma de {col}", frontsize=14)
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

Capital Prestado: Distribucion sesgada hacia la derecha con la mayoria de prestamos concentrados en valores bajos. Presencia de outliers sugiere diferentes segmentos de clientes

Plazo meses: Distribucion relativamengte uniforme con concentracion de plazos estandar (12, 23, 36 meses). En terminos de cumplimieto, loscreditos de corto plazo son mas confiables, y el riesgo aumenta en mediano y largo plazo.

Edad Cliente: Distribucion normal con ligero sesgo hacia edades mayores. Concentracion princial entre 30-50 años, tipico de poblacion creiticiamente activa.

Salario Cliente: Fuerte concentracion en salarios bajos con cola larga hacia valores altos. Patron tipico de distribucion de ingresos en poblacion general.

Total Otros Prestamos: Mayoria de clientes sin otros prestamos, con casos aislados de exposiciones multiples muy altas. Sugiere portafolio principalmente de primeros creditos.

Puntaje: Altamente concentrado en puntajes entre 80-100.

Puntaje Datacreidito: Similar al puntaje anterior con mayor concentracion en 800.

Cantidad Creditos Vigentes: Fuerte concentracion entre 0-10. Pocos clientes con multiples exposiciones, indicando gestion conservadora de riesgo.

Huella Consulta: Distribucion sesgada hacia la derecha alta concentracion de personas entre 0 - 5. 

Saldo Mora: Mayoria de clientes sin mora (valores en cero), con casos aislados de moras significativas.

Saldo Total: Gran variabilidad con mediana muy inferior a la medida, confirmando concentracion en saldos bajos con outliers extremos.

Saldo Principal: Patron similar al saldo total

Saldo Mora Codeudor: Mayoria absoluta en cero, indicando baja utilizacion de codeudores y/o buen comportamiento de pago conjunto.

Creditos Sector Financiero: Distribucion concentrada en valores bajos, con mayoria de clientes teniendo pocos o ningun credito en sector financiero

Creditos Sectos Cooperativo: Similar al sector financiero, concentracion en valores bajos sugiriendo baja penetracion del secto cooperativo.

Estadisticas Creditos Sector Real: Baja exposicion promedio en sector real, consistente con el perfil de cartera observado.

Promedio Ingreso Datacredito: Distribucion sesgada similar al salrio cliente.



In [ ]:
#   Filtrar solo columnas numericas

num_cols= df.select_dtypes(include=['int64', 'float64']).columns

#   Recorrer y graficar Boxplots

for col in num_cols:
    plt.figure(figsize=(8,4))
    sns.barplot(x=df[col].dropna())
    plt.title(f"Boxplot de {col}", fontsize=14)
    plt.xlabel(col)
    plt.grid(axis="x", linestyle="--", alpha=0,7)
    plt.show()

### Boxplot Capital Prestado
*   Mediana muy baja (~500K) con Q3 de 2M, indicando que 75% de los prestamos son de monto pequeño-mediano. Outliers extremos hasta 50M+ sugieren prestamos corporativos o hipotecarios. La caja compacta confirma concentracion en segmentos retail.

### Boxplot Edad Cliente
*   Distribucion simetrica con mediana ~40 años. Ranfo inrcuartilico normal (30-50 años) sin outliers extremos, confirmando poblacion crediticia estandar. Los whiskers se extienden apropiadamente hasta los limites naturales de edad crediticia.

### Boxplot Salario Cliente
*   Mediana baja (~2M) con Q3 alrededor de 4M, mostrando concentracion en ingresos medios-bajos. Outliers extremos hasta 50M+ indican ejecutivos o empresarios. La asimetria hacia arriba es tipica de distribuciones de ingreso.

### Boxplot Total Otros Prestamos
*   Mediana en cero confirma que mayoria de clientes no tienen otros prestamos. Q3 bajo indica que incluso al 75% superior tiene exposiciones limitadas. Outliers significativos sugieren clientes con multiples obligaciones financieras.

### Boxplot Couta Pactada
*   Mediana baja (~200K) consistente con ingresos y montos prestados observados. La proporcion couta/ingreso implicita parece conservadora. Outliers extremos corresponden a los prestamos de mayor monto identificados previamente.

### Boxplot Cantidad Creditos Vigentes
*   Mediana en 1-2 creditos con Q3 alrededor de 3, indicando gestion conservadora de exposicion multiple. Pocos outliers con muchos creditos sugieren casos especiales o clientes premium con mayor capacidad de endeudamiento.

### Boxplot Saldos (Variables multiples)
*    Todos muestran patron similar: medianas bajas, alta concentracion de calores pequeños y outliers extremos. Confirma que el portafolio esta dominado por creditos pequeños con casos aislados de exposiciones significativas.

In [ ]:
# Filtrar numericas
num_cols = df.select_dtypes(include=['int64','float64']).columns.drop("Pago_atiempo")

# Generar tablas pivote
for col in num_cols:
    tabla = df.pivot_table(values=col, index="Pago_atiempo", aggfunc=["mean", "median", "min", "max"])
    print(f"\n Tabla pivote para {col} con respecto a Pago_atiempo:")
    print(tabla)

In [ ]:
# Buscar columnas categoricas directamente
categorical_cols = df.select_dtypes(include=["object", "category", "bool"]).columns

for col in categorical_cols:
    plt.figure(figsize=(10, 6))
    df[col].value_counts().plot(kind="bar")
    plt.title(f"Frecuencia de {col}")
    plt.xlabel(col)
    plt.ylabel("Conteo")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    print(f"Values counts for {col}:\n{df[col].value_counts()}\n")
    print("-" * 50)

### Tipo credito
*   Existe una categoria claramente dominante (probablemente consumo o libre inversion).
*   Las demas categorias aparecen en proporciones mucho menores.
*   Esto podria generar desbalance en las clases y requerir tecnicas de balaceo o agrupacion.

### Tipo Laboral
*   Mayoria de clientes son empleados, mientras que los independientes representan a un grupo menor.
*   Podria ser relevante porque los independientes suelen tener mayor probabilidad de incumplimiento.

### Analisis bivariable

Nos interesa revisar cada variable respecto a la objetivo. Para las variables numericas boxplot como hue o columna la variable objetivo. Hisplot con hue=columna objetivo

In [ ]:
# Variables numericas
num_cols =df.select_dtypes(include=["int64", "float64"]).drop(columns=["Pago_atiempo"]).columns

for col in num_cols:
    plt.figure(figsize=(7, 5))
    sns.histplot(data=df, x=col, hue="Pago_atiempo", element="step", stat="density", common_norm=False)
    plt.title(f"Ditribucion de {col} segun Pago_atiempo", fontsize=14)
    plt.xlabel(col)
    plt.ylabel("Densidad")
    plt.grid(axis="y", linesyle="--", alpha=0.7)
    plt.show()

### Capital Prestado vs Pago a Tiempo: 
*   Los clientes que no pagan a tiempo tienden a tener prestamos de montos mas altos. La distribucion muestra mayor dispersion en el grupo de no pagadores, sugiriendo que montos elevados aumentan el riesgo de incumplimiento
### Plazo Meses vs Pago a Tiempo:
*   Los buenos pagadores se concentran en plazos medios (12-36 meses), mientras que los malos pagadores muestran variabilidad. Plazos muy largos parecen asociarse con mayor riesgo de incumplimiento
### Edad Cliente vs Pago a Tiempo:
*   Diferencia notable: clientes que pagan a tiempo tienden a ser mayores (concentracion 35-50 años). Los mas jovenes muestran mayor propension al incumplimiento, confirmando edad como factor de riesgo.
### Salario Cliente vs Pago a Tiempo:
*   Los buenos pagadores muestran distribucion mas concentrada en ingresos medios-altos. Los malos pagadores tienen mayor dispersion, incluyendo casos de ingresos altos que no pagan, sugiriendo otros factores de riesgo.
### Total Otros Prestamos vs Pago a Tiempo:
* Clara diferenciacion: malos pagadores tienen significativamente mas exposicion en otros prestamos. Alta correlacion entre sobreendeudamiento y riesgo de incumplimiento.
### Couta pactada vs Pago a Tiempo:
*   Los buenos pagadores se concentran en coutas bajas-medias. Coutas muy altas se asoscian con mayor riesgo, probablemente por compromiso excesivo de la capacidad de pago.

In [ ]:
sns.pairplot(df, vars=["capital_prestado", "salario_cliente", "puntaje", "edad_cliente"], 
             hue="Pago_atiempo", diag_kind="kde", corner=True)
plt.suptitle("Pairplot de variables numericas con respecto a Pago_atiempo", y=1.02, fontsize=14)
plt.show()

In [ ]:
pd.crosstab([df["tipo_laboral"], df["tipo_credito"]], df["Pago_atiempo"], normalize="index") * 100

In [ ]:
pd.crosstab([df["tendencia_ingresos"], df["tipo_laboral"]], df["Pago_atiempo"], normalize="index") * 100